BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits. 
It is assumed that a local llm is running on your system and has been invoked with this code, for instance here llama 3.2 is engaged.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
#from dotenv import load_dotenv #running llama locally, hence this wont be needed
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

Initialize and constants in case of using Open AI API

"load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()"

In [3]:
#local llm connection
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [4]:
# A class to represent a Webpage

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
#ed.links

First step: Have Llama3.2 figure out which links are relevant

Use a call to llama3.2 to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

#print(link_system_prompt)

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, legal, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

#print(get_links_user_prompt(ed))

In [8]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [9]:
#edwarddonner = Website("https://edwarddonner.com")
#edwarddonner.links

#get_links("https://edwarddonner.com")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [11]:
#print(get_all_details("https://edwarddonner.com"))

In [12]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [14]:
#get_brochure_user_prompt("Ed-Donner", "https://edwarddonner.com")

In [15]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
create_brochure("Ed-Donner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Company page (homepage)', 'url': 'https://edwarddonner.com/'}, {'type': 'Posts', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'About the author', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}]}


## Ed-Donner Brochure
### About Our Mission

Welcome to Ed-Donner, a company dedicated to revolutionizing the world of work and talent management. We are driven by a simple yet profound mission: to help people discover their potential and pursue their reason for being.

At Ed-Donner, we believe that AI has the power to make a massive, positive impact on society. Our team is passionate about applying this technology to real-world problems, particularly in the realm of hiring and talent management.

### Our Story

Our journey began with our AI startup, untapt, which was founded in 2013 by ourselves, Ed. We specialized in building talent marketplaces and data science software for recruitment firms, focusing on tech roles in financial services. After a successful 20-year career, we sold untapt to GQR's parent company in 2021.

### Our Current Venture

With the acquisition of untapt came the birth of Nebula.io. We have patented our revolutionary matching model and developed an award-winning platform that has already made a significant impact on the industry we serve.

Nebula.io is helping recruiters source, understand, engage, and manage talent using Generative AI and other forms of machine learning. Our long-term goal is to empower individuals to discover their Ikigai – a concept that describes the sweet spot where one's passion for work meets their skills.

### Connect Four

One of our flagship features is Outsmart, an arena that pits LLMs against each other in a battle of diplomacy and deviousness. This platform allows us to test our LLMs in real-world settings, enabling us to identify areas for improvement and push the boundaries of what's possible with AI.

### Join Our Community

We're on a mission to make a tangible difference in people's lives. If you share our passion for making work fulfilling, engaging, and purpose-driven, we'd love to connect with you.

- Follow us: LinkedIn | Twitter | Facebook
- Subscribe to our newsletter: [link]
- Get in touch: ed@edwarddonner.com

### Opportunities to Join Us

We're a dynamic team of pioneers seeking talented individuals who share our vision. Explore our career page to find out about current openings and how you can be part of shaping the future of work.

[Link to career page]

### Thank You!

Thank you for being part of the Ed-Donner journey. Let's connect, collaborate, and make a difference together!

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("Ed-Donner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'Company page', 'url': 'https://edwarddonner.com'}, {'type': 'About page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Blogger/Posts page', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}]}


# Ed-Donner Brochure

## Mission & Purpose

At Ed-Donner, our mission is to empower individuals to discover their potential and pursue their purpose, fostering human prosperity through the innovative application of Artificial Intelligence (AI) in the field of talent sourcing and management.

## Company Culture

Our company embodies a forward-thinking culture that values experimentation, creativity, and collaboration. Our founders and team members share a passion for making a positive impact by applying AI to real-world problems, particularly in the realm of hiring and talent management.

Led by our Co-Founder and CTO, Ed, who brings extensive experience as a leader in AI startups, including the acquisition of his previous company in 2021. Our team is composed of like-minded individuals who are equally committed to utilizing AI for the greater good.

## Applications & Expertise

Ed-Donner specializes in applying Generative AI and machine learning to help recruiters source, understand, engage, and manage talent with unparalleled accuracy and speed. We have patented our unique matching model that operates on a vast array of keywords:

**Key Services:**

* Talent sourcing
* Role matching without traditional keywords
* Engagement and management capabilities for HR software

## Industry Expertise & Community

We are actively engaged in the AI community, hosting events like **Connect Four** (an arena pitting LLMs against each other in a battle of diplomacy and deviousness) to showcase our cutting-edge technology. Additionally, we offer various courses to help professionals enhance their knowledge and skills in LLM expert leadership, including [The Complete Agentic AI Engineering Course](January 23, 2025).

**Stay Connected:**

- Follow us on LinkedIn
- Subscribe to our newsletter

## Vision for the Future

Our long-term goal is to contribute significantly to human prosperity by increasing the levels of job satisfaction and fulfillment through precision role matching.

# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")